In [3]:
# Importação das bibliotecas
import pandas as pd
import numpy as np
import json, os, uuid
from google_play_scraper import Sort, reviews_all, reviews, app
from app_store_scraper import AppStore
import random

In [4]:
# Versões dos pacotes usados neste notebook
%reload_ext watermark
%watermark -a "Grupo J - 1DTSO" --iversions

Author: Grupo J - 1DTSO

numpy : 1.23.4
json  : 2.0.9
pandas: 1.5.0



In [2]:
# Reviews do Google Play Store
g_reviews = reviews_all(
    'br.com.petz',
    sleep_milliseconds = 0,
    lang = 'pt',
    sort=Sort.NEWEST,
    count = 500,
    filter_score_with = None
)

In [5]:
# Reviews da Apple Store
a_reviews = AppStore('br','petz-loja-online-para-seu-pet','1250491446')
a_reviews.review(how_many=15000)

2022-10-20 13:43:46,957 [INFO] Base - Initialised: AppStore('br', 'petz-loja-online-para-seu-pet', 1250491446)
2022-10-20 13:43:46,958 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/br/app/petz-loja-online-para-seu-pet/id1250491446
2022-10-20 13:43:52,024 [INFO] Base - [id:1250491446] Fetched 440 reviews (440 fetched in total)
2022-10-20 13:43:57,467 [INFO] Base - [id:1250491446] Fetched 880 reviews (880 fetched in total)
2022-10-20 13:43:59,483 [INFO] Base - [id:1250491446] Fetched 1038 reviews (1038 fetched in total)


In [6]:
g_df = pd.DataFrame(np.array(g_reviews),columns=['review'])
g_df2 = g_df.join(pd.DataFrame(g_df.pop('review').tolist()))
g_df2
g_df2.drop(columns={'userImage', 'reviewCreatedVersion'},inplace = True)
g_df2.rename(columns= {'score': 'rating','userName': 'user_name', 'reviewId': 'review_id', 'content': 'review_description', 'at': 'review_date', 'replyContent': 'developer_response', 'repliedAt': 'developer_response_date', 'thumbsUpCount': 'thumbs_up'},inplace = True)
g_df2.insert(loc=0, column='source', value='Google Play')
g_df2.insert(loc=3, column='review_title', value=None)
g_df2['laguage_code'] = 'pt'
g_df2['country_code'] = 'us'

In [7]:
g_df2.shape

(30780, 12)

In [8]:
g_df2.columns

Index(['source', 'review_id', 'user_name', 'review_title',
       'review_description', 'rating', 'thumbs_up', 'review_date',
       'developer_response', 'developer_response_date', 'laguage_code',
       'country_code'],
      dtype='object')

In [9]:
a_df = pd.DataFrame(np.array(a_reviews.reviews),columns=['review'])
a_df2 = a_df.join(pd.DataFrame(a_df.pop('review').tolist()))
a_df2.shape

(1038, 7)

In [10]:
a_df2.columns

Index(['date', 'developerResponse', 'rating', 'title', 'review', 'userName',
       'isEdited'],
      dtype='object')

In [28]:
#Transformando a consulta da App Store em DataFrame
#Removendo coluna isEdited e renomeando abaixo para ficar no mesmo padrão de nome do google play Store
a_df2.drop(columns={'isEdited'},inplace = True)
a_df2.insert(loc=0, column='source', value='App Store')
a_df2['developer_response_date'] = None
a_df2['thumbs_up'] = None
a_df2['laguage_code'] = 'en'
a_df2['country_code'] = 'us'

#Renomeando
a_df2.insert(loc=1, column='review_id', value=[uuid.uuid4() for _ in range(len(a_df2.index))])
#a_df2.rename(columns= {'review': 'review_description','userName': 'user_name', 'date': 'review_date','title': 'review_title', 'developerResponse': 'developer_response'},inplace = True)
a_df2.rename(columns= {'review': 'review_description','userName': 'user_name', 'date': 'review_date','title': 'review_title', 'developerResponse': 'developer_response'},inplace = True)
a_df2 = a_df2.where(pd.notnull(a_df2), None)

KeyError: "['isEdited'] not found in axis"

In [ ]:
a_df2

In [29]:
# Trasnformando os dois dataframes no objeto result
result = pd.concat([g_df2,a_df2])
result

,source,review_id,user_name,review_title,review_description,rating,thumbs_up,review_date,developer_response,developer_response_date,laguage_code,country_code
0,Google Play,2815a902-86ca-4b51-9488-349d8b824164,Viviane Araujo,None,"App fácil de manusear, entrega rápida e bons p...",5,0,2022-10-19 13:07:57,"Estamos empenhados em manter sua satisfação, V...",2022-10-19 19:30:25,pt,us
1,Google Play,4db7136d-44e4-44f4-9b7f-599b23b0ae3e,Edu Lopes,None,Muito bom,5,0,2022-10-19 12:54:19,Obrigada por compartilhar sua percepção com a ...,2022-10-19 19:16:20,pt,us
2,Google Play,279524e3-cd50-4cf2-b19f-8880d0692349,Mariana Gama,None,"Péssimo atendimento, péssimo app fiz uma compr...",1,0,2022-10-19 12:39:36,Temos interesse em te ajudar a respeito desta ...,2022-10-19 19:15:11,pt,us
3,Google Play,2eb8eb15-1a26-4282-9f14-85b998aff9a7,Claudia Regina,None,Ótimo,5,0,2022-10-19 12:10:45,"Agradecemos pela avaliação, Claudia. O seu fee...",2022-10-19 13:05:13,pt,us
4,Google Play,85cfb4b4-f0ea-484c-9ff4-e8e7ce480f26,Juliana Silveira,None,Excelente,5,0,2022-10-19 10:19:24,"Agradecemos pela avaliação, Juliana. O seu fee...",2022-10-19 19:13:02,pt,us
...,...,...,...,...,...,...,...,...,...,...,...,...
1033,App Store,1a7d2f10-487d-496f-a0a2-d22490dea4a4,maicon tatui,App não cadastra direito e trava a todo momento,Boa tarde esse app não esta funcionando a temp...,1,None,2018-10-09 16:20:18,"{'id': 6387542, 'body': 'Atendendo a pedidos, ...",NaT,en,us
1034,App Store,1f12083d-8c09-428e-8d1f-445a6f97b9c2,Maquentar T.,Pouco útil,"O app tem pouca utilidade, para quem tem um ca...",2,None,2018-07-25 13:08:38,"{'id': 6387546, 'body': 'Atendendo a pedidos, ...",NaT,en,us
1035,App Store,e89a2b63-4dd2-483c-961e-a4d22977982c,ThiagoRDias,Sensacional!,Passeio com meus petz. Baita ideia.,5,None,2018-03-15 20:44:32,"{'id': 9407325, 'body': 'Olá, Thiago! Muito ob...",NaT,en,us
1036,App Store,3ae5cb7d-f9f9-484b-9dd6-ebab43a896f8,Priscila CSilva,Não consigo cadastrar,App pouco inteligente. Meu CPF foi cadastrado ...,1,None,2018-03-25 21:03:05,"{'id': 6387550, 'body': 'Atendendo a pedidos, ...",NaT,en,us


In [21]:
consulta = result.drop(columns={'developer_response','developer_response_date', 'laguage_code', 'country_code' })

In [22]:
consulta.tail()

,source,review_id,user_name,review_title,review_description,rating,thumbs_up,review_date
1033,App Store,1a7d2f10-487d-496f-a0a2-d22490dea4a4,maicon tatui,App não cadastra direito e trava a todo momento,Boa tarde esse app não esta funcionando a temp...,1,None,2018-10-09 16:20:18
1034,App Store,1f12083d-8c09-428e-8d1f-445a6f97b9c2,Maquentar T.,Pouco útil,"O app tem pouca utilidade, para quem tem um ca...",2,None,2018-07-25 13:08:38
1035,App Store,e89a2b63-4dd2-483c-961e-a4d22977982c,ThiagoRDias,Sensacional!,Passeio com meus petz. Baita ideia.,5,None,2018-03-15 20:44:32
1036,App Store,3ae5cb7d-f9f9-484b-9dd6-ebab43a896f8,Priscila CSilva,Não consigo cadastrar,App pouco inteligente. Meu CPF foi cadastrado ...,1,None,2018-03-25 21:03:05
1037,App Store,a2b1b2a2-83fa-472d-be11-7fac8f4aa43d,G.Marques,Funcionalidades como site,Seria interessante que o app tivesse as funcio...,1,None,2018-03-20 17:42:50


In [23]:
consulta.dtypes

source                        object
review_id                     object
user_name                     object
review_title                  object
review_description            object
rating                         int64
thumbs_up                     object
review_date           datetime64[ns]
dtype: object

In [25]:
# Extração para Excel
consulta.to_excel("Petz_stores.xlsx",
             sheet_name='Sheet_name_1')

In [26]:
consulta

,source,review_id,user_name,review_title,review_description,rating,thumbs_up,review_date
0,Google Play,2815a902-86ca-4b51-9488-349d8b824164,Viviane Araujo,None,"App fácil de manusear, entrega rápida e bons p...",5,0,2022-10-19 13:07:57
1,Google Play,4db7136d-44e4-44f4-9b7f-599b23b0ae3e,Edu Lopes,None,Muito bom,5,0,2022-10-19 12:54:19
2,Google Play,279524e3-cd50-4cf2-b19f-8880d0692349,Mariana Gama,None,"Péssimo atendimento, péssimo app fiz uma compr...",1,0,2022-10-19 12:39:36
3,Google Play,2eb8eb15-1a26-4282-9f14-85b998aff9a7,Claudia Regina,None,Ótimo,5,0,2022-10-19 12:10:45
4,Google Play,85cfb4b4-f0ea-484c-9ff4-e8e7ce480f26,Juliana Silveira,None,Excelente,5,0,2022-10-19 10:19:24
...,...,...,...,...,...,...,...,...
1033,App Store,1a7d2f10-487d-496f-a0a2-d22490dea4a4,maicon tatui,App não cadastra direito e trava a todo momento,Boa tarde esse app não esta funcionando a temp...,1,None,2018-10-09 16:20:18
1034,App Store,1f12083d-8c09-428e-8d1f-445a6f97b9c2,Maquentar T.,Pouco útil,"O app tem pouca utilidade, para quem tem um ca...",2,None,2018-07-25 13:08:38
1035,App Store,e89a2b63-4dd2-483c-961e-a4d22977982c,ThiagoRDias,Sensacional!,Passeio com meus petz. Baita ideia.,5,None,2018-03-15 20:44:32
1036,App Store,3ae5cb7d-f9f9-484b-9dd6-ebab43a896f8,Priscila CSilva,Não consigo cadastrar,App pouco inteligente. Meu CPF foi cadastrado ...,1,None,2018-03-25 21:03:05
